In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as ptch
%matplotlib notebook
from os import walk
import tensorflow as tf
from tensorflow.keras.models import load_model
from tensorflow.keras import metrics
import pickle

Loading the dataset and normalizing

In [ ]:
# Laod  data 
test_input = pd.read_pickle('DATA/test_input')
test_output = pd.read_pickle('DATA/test_output')
index = test_input.columns

# Normlaization of input data
# Data normalization according to training dataset/ model
filehandler = open('Weights/Norm', 'rb') 
std_scaler = pickle.load(filehandler)
inputn = pd.DataFrame(std_scaler.transform(test_input), columns=test_input.columns) 
# inputn is still a daraframe with numeric index
outputn = test_output/10**6  #  change of units, outputn is still a daraframe with time index

Define the models to be tested

In [ ]:
include_wave = 'no'
modelIDs = [0, 1, 2, 3, 4, 5]
# modelIDs = [3]
# (0 for SCADA only, 1 for SCADA+Acc17, 2 for SCADA+Acc38, 3 for SCADA+Acc77, 
# 4 for SCADA+Acc17&38, 5 for SCADA+Acc17&38&77 

Err_Mtl1 = np.zeros([len(modelIDs)])
Err_Mtn1 = np.zeros([len(modelIDs)])
for i in range(len(modelIDs)):
    modelID = modelIDs[i]
    
    # Retrive features based on the modelID
    index1 = pd.core.indexes.base.Index([]) # create a blank index array
    if include_wave == 'yes': 
        index1 = index1.append(index[0:3])
    if modelID == 1: # Acc17
        index1 = index1.append(index[[3,4,9,10,15,16]])
    if modelID == 2: # Acc38
        index1 = index1.append(index[[5,6,11,12,17,18]])
    if modelID == 3: # Acc77
        index1 = index1.append(index[[7,8,13,14,19,20]])
    if modelID == 4: # Acc17&38
        index1 = index1.append(index[[3,4,5,6,9,10,11,12,15,16,17,18]])   
    if modelID == 5: # Acc17&38&77
        index1 = index1.append(index[3:21])
    index1 = index1.append(index[21:]) # SCADA
    X = inputn[index1].values
    Y = outputn.values
    
    # Loading the model to evaluate
    VSnet = load_model('Weights/01_SensorPlacementTest/DNNModel%s_IncWave%s.h5' % (modelID, include_wave))
    
    # Prediction and Compute Error
    nn_output = VSnet.predict(X)
    Err_Mtl1[i] = np.mean(abs((Y[:,0]- nn_output[:,0])/Y[:,0]))*100
    Err_Mtn1[i] = np.mean(abs((Y[:,1]- nn_output[:,1])/Y[:,1]))*100   

In [ ]:
print(Err_Mtl1)
print(Err_Mtn1)

In [ ]:
include_wave = 'yes'
modelIDs = [0, 1, 2, 3, 4, 5]
# modelIDs = [3]
# (0 for SCADA only, 1 for SCADA+Acc17, 2 for SCADA+Acc38, 3 for SCADA+Acc77, 
# 4 for SCADA+Acc17&38, 5 for SCADA+Acc17&38&77 

Err_Mtl2 = np.zeros([len(modelIDs)])
Err_Mtn2 = np.zeros([len(modelIDs)])
for i in range(len(modelIDs)):
    modelID = modelIDs[i]
    
    # Retrive features based on the modelID
    index1 = pd.core.indexes.base.Index([]) # create a blank index array
    if include_wave == 'yes': 
        index1 = index1.append(index[0:3])
    if modelID == 1: # Acc17
        index1 = index1.append(index[[3,4,9,10,15,16]])
    if modelID == 2: # Acc38
        index1 = index1.append(index[[5,6,11,12,17,18]])
    if modelID == 3: # Acc77
        index1 = index1.append(index[[7,8,13,14,19,20]])
    if modelID == 4: # Acc17&38
        index1 = index1.append(index[[3,4,5,6,9,10,11,12,15,16,17,18]])   
    if modelID == 5: # Acc17&38&77
        index1 = index1.append(index[3:21])
    index1 = index1.append(index[21:]) # SCADA
    X = inputn[index1].values
    Y = outputn.values
    
    # Loading the model to evaluate
    VSnet = load_model('Weights/01_SensorPlacementTest/DNNModel%s_IncWave%s.h5' % (modelID, include_wave))
    
    # Prediction and Compute Error
    nn_output = VSnet.predict(X)
    Err_Mtl2[i] = np.mean(abs((Y[:,0]- nn_output[:,0])/Y[:,0]))*100
    Err_Mtn2[i] = np.mean(abs((Y[:,1]- nn_output[:,1])/Y[:,1]))*100   

In [ ]:
Models2 = ['SCADA+Wave', 'SCADA+Wave+Acc017', 'SCADA+Wave+Acc038', 'SCADA+Wave+Acc077', 
              'SCADA+Wave+Acc017,038', 'SCADA+Wave+Acc017,038,077']
Models1 = ['SCADA', 'SCADA+Acc017', 'SCADA+Acc038', 'SCADA+Acc077', 
              'SCADA+Acc017,038', 'SCADA+Acc017,038,077']

cm = 1/2.54  # centimeters in inches
plt.rcParams["font.family"] = "Times New Roman"
plt.rcParams["font.size"] = 9

x1 = np.arange(6)*3+0.5
x2 = np.arange(6)*3+1.0
x3 = np.arange(6)*3+1.65
x4 = np.arange(6)*3+2.15
fig, ax = plt.subplots(1, figsize=(17*cm, 8.5*cm), sharey='row', dpi=80, facecolor='w', edgecolor='k')
plt.subplots_adjust(left=0.07, right=.99, top=0.95, bottom=0.4, hspace = 0.65, wspace=0.15)
# ax.bar(x1, Err_Mtl1, color = '#6baed6', width=0.5, align='center', label = "DEM$_{tl}$ (Without Wave)")
# ax.bar(x2, Err_Mtn1,  color = '#08306b', width=0.5, align='center', label = "DEM$_{tn}$ (With Wave)")
# ax.bar(x3, Err_Mtl2, color = '#bdbdbd', width=0.5, align='center', label = "DEM$_{tl}$ (Without Wave)")
# ax.bar(x4, Err_Mtn2,  color = '#525252', width=0.5, align='center', label = "DEM$_{tn}$ (With Wave)")
ax.bar(x1, Err_Mtl1, color = '#6baed6', width=0.5, align='center', label = "DEM$_{tl}$")
ax.bar(x2, Err_Mtn1,  color = '#08306b', width=0.5, align='center', label = "DEM$_{tn}$")
ax.bar(x3, Err_Mtl2, color = '#6baed6', width=0.5, align='center')
ax.bar(x4, Err_Mtn2,  color = '#08306b', width=0.5, align='center')
ax.set_ylabel('Test error (%)')
ax.set_xticks(np.concatenate((x1+0.25, x3+0.25))) 
ax.set_xticklabels(np.concatenate((Models1, Models2)) ,horizontalalignment= 'right', rotation=45)
ax.set_ylim([0, 28])
def add_value_label(x_list,y_list):
    for i in range(1, len(x_list)+1):
        plt.text(x_list[i-1],y_list[i-1]+0.5,y_list[i-1], ha="center", fontsize = 8)

add_value_label(x1,np.around(Err_Mtl1,1))
add_value_label(x2,np.around(Err_Mtn1,1))
add_value_label(x3,np.around(Err_Mtl2,1))
add_value_label(x4,np.around(Err_Mtn2,1))

ax.add_patch(ptch.Rectangle((9.15,0),1.2,10, 
                        fill=False,
                        color="#a50f15",
                       linewidth=2))

ax.legend(loc ="upper right", fontsize=9, ncol=3)
# plt.grid(axis='y', color='0.95')
fig.savefig('Figures/01_SensorPlacementTest/DNNErrors_merge.pdf')